In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm

In [2]:
# Chuvash alphabet
chuvash_alpahbet = ['А', 'а', 'Ӑ', 'ӑ', 'Б', 'б', 'В', 'в', 'Г', 'г', 'Д', 'д',
                    'Е', 'е', 'Ё', 'ё', 'Ӗ', 'ӗ', 'Ж', 'ж', 'З', 'з', 'И', 'и',
                    'Й', 'й', 'К', 'к', 'Л', 'л', 'М', 'м', 'Н', 'н', 'О', 'о',
                    'П', 'п', 'Р', 'р', 'С', 'с', 'Ҫ', 'ҫ', 'Т', 'т', 'У', 'у',
                    'Ӳ', 'ӳ', 'Ф', 'ф', 'Х', 'х', 'Ц', 'ц', 'Ч', 'ч', 'Ш', 'ш',
                    'Щ', 'щ', 'Ъ', 'ъ', 'Ы', 'ы', 'Ь', 'ь', 'Э', 'э', 'Ю', 'ю',
                    'Я', 'я']


# Russian alphabet
russian_alphabet = ['А', 'а', 'Б', 'б', 'В', 'в', 'Г', 'г', 'Д', 'д', 'Е', 'е',
                    'Ё', 'ё', 'Ж', 'ж', 'З', 'з', 'И', 'и', 'Й', 'й', 'К', 'к',
                    'Л', 'л', 'М', 'м', 'Н', 'н', 'О', 'о', 'П', 'п', 'Р', 'р',
                    'С', 'с', 'Т', 'т', 'У', 'у', 'Ф', 'ф', 'Х', 'х', 'Ц', 'ц',
                    'Ч', 'ч', 'Ш', 'ш', 'Щ', 'щ', 'Ъ', 'ъ', 'Ы', 'ы', 'Ь', 'ь',
                    'Э', 'э', 'Ю', 'ю', 'Я', 'я']



char_map = {'Ӑ': 'А',
            'ӑ': 'а',
            'Ӗ': 'Е',
            'ӗ': 'е',
            'Ӳ': 'У',
            'ӳ': 'у',
            'Ҫ': 'С',
            'ҫ': 'с',
            'ĕ': 'е',
            'ă': 'а',
            'ç': 'c',
            'ÿ': 'у',
            'Ӳ': 'У'}

In [3]:
def replace_chars(text, replacement_dict, replace_probability=0.5):
    new_text = []
    for c in text:
        if c in replacement_dict and random.random() < replace_probability:
            new_text.append(replacement_dict[c])
        else:
            new_text.append(c)
    return ''.join(new_text)

def add_similar_rows(csv_path, percentage, replacement_dict):
    df = pd.read_csv(csv_path)
    num_rows = len(df)
    num_new_rows = int(num_rows * percentage / 100)

    random_indices = np.random.randint(0, num_rows, num_new_rows)
    selected_rows = df.iloc[random_indices].copy()

    selected_rows['chv'] = [replace_chars(text, replacement_dict) for text in tqdm(selected_rows['chv'], desc="Replacing characters")]

    return selected_rows  # return only the modified rows

In [4]:
test_original = pd.read_csv('corpus_test.csv')

In [5]:
test_replacement_50 = add_similar_rows('corpus_test.csv', 100, char_map)

Replacing characters: 100%|██████████| 100201/100201 [00:01<00:00, 83708.51it/s]


In [8]:
test_replacement_50.head()

,rus,chv
83708,С помощью Ольги Шевченко я перебрался к Анне Т...,Ольга Шевченко пулӑшнипе эпӗ Карл Либкнехт ура...
25765,Стоит только ночью сесть на паперть на церковн...,"Курас тесен, ҫерле чиркӳ алӑке умне кайса, ҫул..."
28564,— Заявление в письменной форме? — рассмеялся п...,— Хаван ыйтӑвна ҫырсах парас терӗн-и апла? — к...
48050,Низовые его называют чӳпӗк.,Ҫавна анатрисем чӳпӗк теҫсе.
10824,Уже порой она вступала в споры со стариком; от...,Халӗ енте вӑл ашшепеле тавлашава та кӗркелене;...


In [9]:
test_replacement_50.to_csv('test_replacement_50.csv')

In [ ]:
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/test/test_replacement_50.{src}.txt > data/test/test.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/test/test_replacement_50.{tgt}.txt > data/test/test.{tgt}.bpe